
# Clase 12: Predicciones
## DP-100 Azure Data Scientist / DSRP
## Clase 2 Octubre

### Dataset Utilizado: Hotel Booking

https://www.kaggle.com/code/touba7/hotel-booking

In [1]:
from azure.ai.ml import MLClient, Input
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

MODEL_NAME = "dsrp-booking-model"

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

model = ml_client.models.get(name=MODEL_NAME, label="latest")
model

Found the config file in: /config.json


Model({'job_name': 'dsrp-machine-learning-job-6', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'dsrp-booking-model', 'description': None, 'tags': {'alias': 'CHAMPION'}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourceGroups/aml-course-dp100-2024/providers/Microsoft.MachineLearningServices/workspaces/dsrp-aml-dp100/models/dsrp-booking-model/versions/2', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/test1-dsrp-compute/code/Users/miguel.arquez12/dsrp-azure-data-scientist-course/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f1ac1251570>, 'serialize': <msrest.serialization.Serializer object at 0x7f1ac1251cf0>, 'version': '2', 'latest_version': None, 'path': 'azureml://subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourceGroups/aml-course-dp100-2024/workspaces

# 1. Batch Deployment

In [2]:
import uuid

from azure.ai.ml.entities import (
    BatchEndpoint,
    ModelBatchDeployment,
    ModelBatchDeploymentSettings,
    Model,
    AmlCompute,
    Data,
    BatchRetrySettings,
    CodeConfiguration,
    Environment,
)
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction


batch_endpoint_name = f"b-booking-endpoint-" + str(uuid.uuid4())[:8]
endpoint = BatchEndpoint(
    name=batch_endpoint_name,
    description="Modelo de prediccion de cancelaciones",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

BatchEndpoint({'scoring_uri': 'https://b-booking-endpoint-6ae02bd4.eastus.inference.ml.azure.com/jobs', 'openapi_uri': None, 'provisioning_state': 'Succeeded', 'name': 'b-booking-endpoint-6ae02bd4', 'description': 'Modelo de prediccion de cancelaciones', 'tags': {}, 'properties': {'BatchEndpointCreationApiVersion': '2023-10-01', 'azureml.onlineendpointid': '/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourceGroups/aml-course-dp100-2024/providers/Microsoft.MachineLearningServices/workspaces/dsrp-aml-dp100/batchEndpoints/b-booking-endpoint-6ae02bd4'}, 'print_as_yaml': False, 'id': '/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourceGroups/aml-course-dp100-2024/providers/Microsoft.MachineLearningServices/workspaces/dsrp-aml-dp100/batchEndpoints/b-booking-endpoint-6ae02bd4', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/test1-dsrp-compute/code/Users/miguel.arquez12/dsrp-azure-data-scientist-course/notebooks', 'creation_context'

### Compute cluster

In [3]:
compute_name = "dsrp-cluster"
if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    compute_cluster = AmlCompute( 
        name=compute_name, description="amlcompute para despliegues de modelos", min_instances=0, max_instances=1,
        size="Standard_D1"
    )
    ml_client.begin_create_or_update(compute_cluster).result()

### Deployment

In [4]:
deployment = ModelBatchDeployment(
    name="booking-dsp",
    description="DSRP predicciones de cancelaciones de hoteles",
    endpoint_name=endpoint.name,
    model=model,
    compute=compute_name,
    settings=ModelBatchDeploymentSettings(
        instance_count=1,
        max_concurrency_per_instance=2,
        mini_batch_size=10,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
        logging_level="info",
    ),
)

ml_client.batch_deployments.begin_create_or_update(deployment).result()

Class ModelBatchDeploymentSettings: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ModelBatchDeployment: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


BatchDeployment({'provisioning_state': 'Succeeded', 'endpoint_name': 'b-booking-endpoint-6ae02bd4', 'type': None, 'name': 'booking-dsp', 'description': 'DSRP predicciones de cancelaciones de hoteles', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourceGroups/aml-course-dp100-2024/providers/Microsoft.MachineLearningServices/workspaces/dsrp-aml-dp100/batchEndpoints/b-booking-endpoint-6ae02bd4/deployments/booking-dsp', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/test1-dsrp-compute/code/Users/miguel.arquez12/dsrp-azure-data-scientist-course/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f1ac1286980>, 'serialize': <msrest.serialization.Serializer object at 0x7f1ac1286800>, 'model': '/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourceGroups/aml-course-dp100-2024/providers/Microsoft.MachineLearningServices/workspaces/dsrp-aml

### Prueba de Batch deployment

In [6]:
ml_client.data

In [7]:
data_asset = ml_client.data.get("gold-dsrp-booking-prod", version="2")
data_asset

Data({'path': 'azureml://subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourcegroups/aml-course-dp100-2024/workspaces/dsrp-aml-dp100/datastores/workspaceblobstore/paths/UI/2024-10-23_233720_UTC/', 'skip_validation': False, 'mltable_schema_url': None, 'referenced_uris': None, 'type': 'uri_folder', 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'gold-dsrp-booking-prod', 'description': None, 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': '/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourceGroups/aml-course-dp100-2024/providers/Microsoft.MachineLearningServices/workspaces/dsrp-aml-dp100/data/gold-dsrp-booking-prod/versions/2', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/test1-dsrp-compute/code/Users/miguel.arquez12/dsrp-azure-data-scientist-course/notebooks', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f1acbf1bdf0>, 'serialize': <msrest

In [8]:
input = Input(type=AssetTypes.URI_FOLDER, path=data_asset.id)
job = ml_client.batch_endpoints.invoke(
    deployment_name=deployment.name, endpoint_name=endpoint.name, input=input
)

In [10]:
scoring_job = list(ml_client.jobs.list(parent_job_name=job.name))[0]
scoring_job

Experiment,Name,Type,Status,Details Page
05ad49f6-2042-4a26-81a5-cc58fb74cfed,9a56211c-80c0-4bb5-8cc8-a4fa70f282e8,base,Queued,Link to Azure Machine Learning studio


In [10]:
ml_client.jobs.download(name=scoring_job.name, download_path=".", output_name="score")

In [11]:
import pandas as pd

score = pd.read_csv(
    "named-outputs/score/predictions.csv", names=["row", "prediction", "file"]
)
score

,row,prediction,file
0,0,1,feature_engineering_data_PROCESSED.csv
1,1,1,feature_engineering_data_PROCESSED.csv
2,2,0,feature_engineering_data_PROCESSED.csv
3,3,1,feature_engineering_data_PROCESSED.csv
4,4,1,feature_engineering_data_PROCESSED.csv
...,...,...,...
119385,119385,1,feature_engineering_data_PROCESSED.csv
119386,119386,1,feature_engineering_data_PROCESSED.csv
119387,119387,0,feature_engineering_data_PROCESSED.csv
119388,119388,1,feature_engineering_data_PROCESSED.csv


# 2. Online Deployment

In [13]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
)


In [14]:
# Creating a unique endpoint name with current datetime to avoid conflicts
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="DSRP DESPLIEGUE ONLINE",
    auth_mode="key",
    tags={"MODEL": "CHAMPION"},
)
ml_client.begin_create_or_update(endpoint).result()


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-10170143609221.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-10170143609221.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-10170143609221', 'description': 'DSRP DESPLIEGUE ONLINE', 'tags': {'MODEL': 'CHAMPION'}, 'properties': {'createdBy': 'miguel arquez abdala', 'createdAt': '2024-10-17T01:43:07.111919+0000', 'lastModifiedAt': '2024-10-17T01:43:07.111919+0000', 'azureml.onlineendpointid': '/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourcegroups/aml-course-dp100-2024/providers/microsoft.machinelearningservices/workspaces/dsrp-aml-dp100/onlineendpoints/endpoint-10170143609221', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oeidp:4b014ae2-0df9-417b-a95d-74c60a07cf6a:dcdf18

In [15]:
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_F4s_v2",
    instance_count=1,
)
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpoint-10170143609221 exists


.....................................................................................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'endpoint-10170143609221', 'type': 'Managed', 'name': 'blue', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/odidp:4b014ae2-0df9-417b-a95d-74c60a07cf6a:0675a150-f04d-44f9-b14d-5fbadeb737a8?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/0f61d6bf-ab3d-4df7-a666-edaf42eff57c/resourceGroups/aml-course-dp100-2024/providers/Microsoft.MachineLearningServices/workspaces/dsrp-aml-dp100/onlineEndpoints/endpoint-10170143609221/deployments/blue', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/test1-dsrp-compute/code/Users/miguel.arquez12/dsrp-azure-data-scientist-course/notebooks', 'creation_context': 

## Test online deployment

In [30]:
import json
import pandas as pd

INDEX_TEST = 901


data_asset = ml_client.data.get("gold-booking-dsrp", version="5")
modeling_dataframe = pd.read_csv(data_asset.path)
modeling_dataframe.head()

_input_file = {
  "input_data": {
    "columns": modeling_dataframe.drop("is_canceled", axis=1).columns.to_list(),
    "data": [
      modeling_dataframe.drop("is_canceled", axis=1).iloc[INDEX_TEST].to_list()
    ],
    "index": [INDEX_TEST]
  }
}

with open('online-deployment-dsrp.json', 'w') as f:
    json.dump(_input_file, f)

In [31]:
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="online-deployment-dsrp.json",
)

'[0]'

In [32]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data = _input_file

body = str.encode(json.dumps(data))

url = 'https://endpoint-10170143609221.eastus.inference.ml.azure.com/score'
# Replace this with the primary/secondary key, AMLToken, or Microsoft Entra ID token for the endpoint
api_key = 'lXI4WsUyXhWla59vahCDfCFv4o7TyNea'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

The request failed with status code: 404
content-type: text/plain
content-length: 506
date: Thu, 17 Oct 2024 02:08:06 GMT
server: azureml-frontdoor
x-request-id: 8cb25be5-bdf4-4e7a-ba7f-fd40389f46b7
connection: close


No valid deployments to route to. Please check that the endpoint has at least one deployment with positive weight values or use a deployment specific header to route. [https://docs.microsoft.com/en-us/azure/machine-learning/how-to-safely-rollout-managed-endpoints#deploy-a-new-model-but-send-it-no-traffic-yet]
Please check this guide to understand why this error code might have been returned 
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-troubleshoot-online-endpoints#http-status-codes



In [27]:
urllib.request.urlopen(req)

HTTPError: HTTP Error 404: Not Found